In [1]:
import os

repo_dir = "nlp-Text-De-toxification"

if os.path.exists(repo_dir):
    print(f"{repo_dir} already exists. Removing it...\n")
    !rm -r {repo_dir}

# Clone the repository from GitHub
!git clone https://github.com/Goshmar/nlp-Text-De-toxification

Cloning into 'nlp-Text-De-toxification'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 126 (delta 11), reused 0 (delta 0), pack-reused 90
Receiving objects: 100% (126/126), 2.02 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
! pip install -r nlp-Text-De-toxification/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.4/677.4 kB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.

In [ ]:
import sys
sys.path.append('/content/nlp-Text-De-toxification/src/models')

from importlib import reload\
import condbert
from condbert import CondBertRewriter
reload(condbert)

SyntaxError: ignored

In [ ]:
import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from masked_token_predictor_bert import MaskedTokenPredictorBert

In [ ]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pickle
from tqdm.auto import tqdm, trange

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # please change it to e.g. 'cuda:0' if you want to use a GPU
device

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.to(device);

In [ ]:
# Downloading vocab for CondBert model
def load_words_from_file(file_path):
    with open(file_path, "r") as file:
        words = [line.strip() for line in file.readlines()]
    return words

words_root = "/content/nlp-Text-De-toxification/data/external/"
negative_words = load_words_from_file(words_root + "negative-words.txt")
positive_words = load_words_from_file(words_root + "positive-words.txt")

In [ ]:
import pickle

interim_root = '/content/nlp-Text-De-toxification/data/interim/'

with open(interim_root + "/word2coef.pkl", 'rb') as f:
    word2coef = pickle.load(f)

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

import numpy as np
import pandas as pd

token_toxicities = []
with open(interim_root + "token_toxicities.txt", 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio
# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

In [ ]:
reload(condbert)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=None,
)

In [ ]:
predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0)
editor.predictor = predictor

def adjust_logits(logits, label):
    return logits - editor.token_toxicities * 5

predictor.logits_postprocessor = adjust_logits

In [ ]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

In [ ]:
dataset = pd.read_csv("/content/nlp-Text-De-toxification/data/interim/dataset_cropped.csv")
for example in dataset['reference'].sample(3):
    print("------")
    print(example)
    print("-->", editor.replacement_loop(example, verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))
    print("------\n\n")

------
Yeah, you fuckin' cow. No one in this country can ever pronounce my name right.
--> yeah , you ' re goin ' cow . no one in this country can ever pronounce my name right .
------


------
Do you really want to explain it to every asshole who orders a drink?
--> do you really want to explain it to every sane person who orders a drink ?
------


------
'Grinning Monkey', Argo called him and Lin-tse had despised the youth for his cowardice.
--> ' grinning like a rabbit ' , argo called him and lin - tse had once mocked the youth for his " mischief " .
------


